In [24]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 模型融合

- xgb 0.450681 + lgb 0.450017 = 0.4522 

-  xgb 0.450681 * 0.8 + lgb 0.450017 * 0.2 = 0.451317

In [25]:
submit_path = datetime.now().strftime('../submits/%d-%H%M_l_a_x.csv')
submit_path

'../submits/07-2230_l_a_x.csv'

In [26]:
lmodel = pickle.load(open('../dataset/lightgbmmodel.pkl', 'rb'))

In [27]:
xmodel = pickle.load(open('../dataset/xgbmodel.pkl', 'rb'))

In [28]:
test_feature = pd.read_pickle('../dataset/test/all_feature.pkl')
test_feature.shape

(978298, 443)

In [30]:
use_columns = [
    x for x in test_feature.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid', 'orderlabel'
    ]
]

In [31]:
lresult = lmodel.predict_proba(test_feature[use_columns])[:, 1]

In [32]:
xresult = xmodel.predict_proba(test_feature[use_columns])[:, 1]

In [33]:
test_feature['lprob'] = lresult
test_feature['xprob'] = xresult

In [35]:
test_feature['prob'] = test_feature['lprob']  + test_feature['xprob'] 

In [36]:
test_feature.sort_values('prob', ascending=False, inplace=True)

In [37]:
submit = test_feature.drop_duplicates(['orderid'])[['orderid', 'roomid', 'prob']]

In [38]:
submit['orderid'] = 'ORDER_' + submit['orderid'].astype(str)

In [39]:
submit['predict_roomid'] = 'ROOM_' + submit.roomid.astype(str)

In [40]:
submit.to_csv(submit_path, index=False, columns=['orderid', 'predict_roomid'])

In [41]:
!wc -l $submit_path

34633 ../submits/07-2230_l_a_x.csv


In [42]:
!head $submit_path

orderid,predict_roomid
ORDER_107886,ROOM_1769773
ORDER_139093,ROOM_4164603
ORDER_126119,ROOM_7067233
ORDER_113074,ROOM_7978779
ORDER_109243,ROOM_11421623
ORDER_120781,ROOM_394923
ORDER_107558,ROOM_11202563
ORDER_129956,ROOM_11346707
ORDER_120936,ROOM_1226453


In [43]:
test_feature.orderid.unique().shape

(34632,)